In [ ]:
from Functions_For_LIF_analysis import import_lif_file, quick_view_3d_images, cell_counter_3D, display_all_3Dlayers_from_list
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import copy

In [ ]:
#The data used for this 3D image analysis is flawed because of an incorrect experiment analysis(the additional dye channel pkh26
#made the cell membranes permeable and this made the live/dead cell dyes work incorrectly), therefore the ratios of the cells
#are not correct and the experiments will have to be redone, but this code should be applicable in the same way then too
day5_data = import_lif_file(r"data\RASC025 5 day livedeadpkh26 3D.lif")
day10_data = import_lif_file(r"data\RASC025 live-dead-pkh26 10 day 3D.lif")

In [ ]:
quick_view_3d_images(day5_data)

In [ ]:
quick_view_3d_images(day10_data)

In [ ]:
display_all_3Dlayers_from_list(day5_data)

In [ ]:
display_all_3Dlayers_from_list(day10_data)

In [ ]:
#The third channel has a seperate dye, that is not necessary for the live/dead ratio count

data5_data_noCh3 = copy.deepcopy(day5_data)

for image in data5_data_noCh3:
    if len(image['channel_images']) >= 3:
        del image['channel_images'][2]

        if len(image['variable_names']) >= 3:
            del image['variable_names'][2]

In [ ]:
data10_data_noCh3 = copy.deepcopy(day10_data)

for image in data10_data_noCh3:
    if len(image['channel_images']) >= 3:
        del image['channel_images'][2]

        if len(image['variable_names']) >= 3:
            del image['variable_names'][2]

In [ ]:
#Removes images of poor quality
day5_clean = copy.deepcopy(data5_data_noCh3)

if len(day5_clean) > 2:
    del day5_clean[2]

In [ ]:
display_all_3Dlayers_from_list(day5_clean)

In [ ]:
#Removes images of poor quality
day10_clean = copy.deepcopy(data10_data_noCh3)

if len(day10_clean) > 3:
    del day10_clean[2:4]  

In [ ]:
display_all_3Dlayers_from_list(day10_clean)

In [ ]:
#In the cell count I take only part of the layers to minimize the effect of artifacts caused by laser flare and other phenomena at different layers
result_dict = {}

for image_data in day5_clean:
    image_name = image_data["image_name"]
    channel_images = image_data["channel_images"]
    
    for channel_index, np_image in enumerate(channel_images[:2]):
        selected_slices = np_image[10:20]
        stacked_image = np.stack(selected_slices, axis=0)
        cell_count = cell_counter_3D(f"{image_name}_channel_{channel_index + 1}", stacked_image)
        result_dict[f"{image_name}_channel_{channel_index + 1}"] = cell_count

print(result_dict)

In [ ]:
#In the cell count I take only part of the layers to minimize the effect of artifacts caused by laser flare and other
#phenomena at different layers  and also to reduce the computational time.
result_dict2 = {}

for image_data in day10_clean: 
    image_name = image_data["image_name"]
    channel_images = image_data["channel_images"]
    
    for channel_index, np_image in enumerate(channel_images[:2]):
        selected_slices = np_image[10:20]
        stacked_image = np.stack(selected_slices, axis=0)
        cell_count = cell_counter_3D(f"{image_name}_channel_{channel_index + 1}", stacked_image)
        result_dict2[f"{image_name}_channel_{channel_index + 1}"] = cell_count

print(result_dict2)

In [ ]:
def create_dataframe_from_dict(result_dict):
    # Split the dictionary into two based on channel
    channel_1_dict = {key: value for key, value in result_dict.items() if 'channel_1' in key}
    channel_2_dict = {key: value for key, value in result_dict.items() if 'channel_2' in key}

    # Process to align the data correctly
    # Extract the base image name (without channel part)
    channel_1_names = [key.rsplit('_channel_1', 1)[0] for key in channel_1_dict.keys()]
    channel_2_names = [key.rsplit('_channel_2', 1)[0] for key in channel_2_dict.keys()]

    # Combine and sort the unique names
    all_image_names = sorted(set(channel_1_names) | set(channel_2_names))

    # Prepare the data for the dataframe
    data = {
        'Channel 1': [channel_1_dict.get(name + '_channel_1', 0) for name in all_image_names],
        'Channel 2': [channel_2_dict.get(name + '_channel_2', 0) for name in all_image_names]
    }

    # Create and return the dataframe
    df = pd.DataFrame(data, index=all_image_names)
    return df

# Assuming result_dict and result_dict2 are defined as per your input
df1 = create_dataframe_from_dict(result_dict)
df2 = create_dataframe_from_dict(result_dict2)

# Display the dataframes
df2

In [ ]:
#Live cells are in the second channel and the dead cells are in the first one
def calculate_live_dead_ratio(df):
    total_channel_1 = df['Channel 1'].sum()
    total_channel_2 = df['Channel 2'].sum()
    ratio = total_channel_1 / total_channel_2 if total_channel_2 != 0 else 0
    return ratio

ratio_day_5 = calculate_live_dead_ratio(df1)
ratio_day_10 = calculate_live_dead_ratio(df2)

live_dead_ratio_df = pd.DataFrame({'live/dead ratio': [ratio_day_5, ratio_day_10]}, index=['Day 5', 'Day 10'])

print(live_dead_ratio_df)

In [ ]:
def calculate_individual_ratios(df):
    ratios = df['Channel 1'] / df['Channel 2'].replace(0, np.nan) 
    return ratios.dropna() 


individual_ratios_day_5 = calculate_individual_ratios(df1)
individual_ratios_day_10 = calculate_individual_ratios(df2)


mean_ratio_day_5 = individual_ratios_day_5.mean()
std_error_day_5 = individual_ratios_day_5.std() / np.sqrt(len(individual_ratios_day_5))

mean_ratio_day_10 = individual_ratios_day_10.mean()
std_error_day_10 = individual_ratios_day_10.std() / np.sqrt(len(individual_ratios_day_10))


live_dead_ratio_df = pd.DataFrame({'live/dead ratio': [mean_ratio_day_5, mean_ratio_day_10],
                                   'std_error': [std_error_day_5, std_error_day_10]},
                                   index=['Day 5', 'Day 10'])


plt.figure(figsize=(8, 6))
plt.bar(live_dead_ratio_df.index, live_dead_ratio_df['live/dead ratio'], 
        yerr=live_dead_ratio_df['std_error'], capsize=5, color='tab:blue')
plt.xlabel('Day')
plt.ylabel('Live/Dead Ratio')
plt.title('Live/Dead Ratio for Day 5 and Day 10 with Error Bars')
plt.xticks(rotation=45)
plt.show()

live_dead_ratio_df